# Chapter 5 : Reduction of Multiple Subsystems
---

In [1]:
import sys

if (path := "C:/Users/Tom/pycharm-projects/python-control") not in sys.path:
    sys.path.append(path)

import sympy as sp

from python_control import s, TransferFunction, SecondOrderSystem, SignalFlowGraph

## 5.2 : Block Diagrams

### Example 1 from Appendix B (ch5apB1.m)

Find the total transfer function of the pitch control system for the UFSS (Unmanned Free-Swimming Submersible Vehicle).

![pitch control system](./images/pitch_control_system.png)

In [2]:
K1 = K2 = 1

**Solution via algebraic operations**

Pitch gain

In [3]:
G1 = TransferFunction(-K1)
G1.expr

-1.00000000000000

Elevator actuator

In [4]:
G2 = TransferFunction.from_coefficients(num=[2], den=[1, 2])
G2.expr

2.0/(1.0*s + 2.0)

Vehicle dynamics

In [5]:
G3 = TransferFunction(-0.125 * (s + 0.435) / ((s + 1.23) * (s**2 + 0.226 * s + 0.0169)))
G3.expr

(-0.125*s - 0.054375)/(1.0*s**3 + 1.456*s**2 + 0.29488*s + 0.020787)

Pitch rate sensor

In [6]:
H1 = TransferFunction(-K2 * s)
H1.expr

-1.0*s

Block diagram reduction:

In [7]:
G4 = G2 * G3
G5 = G4.feedback(H1)
Ge = G1 * G5
Ge.expr

(0.25*s + 0.10875)/(1.0*s**4 + 3.456*s**3 + 3.45688*s**2 + 0.719297*s + 0.041574)

In [8]:
T = Ge.feedback(TransferFunction(1))
T.expr

(0.25*s + 0.10875)/(1.0*s**4 + 3.456*s**3 + 3.45688*s**2 + 0.969297*s + 0.150324)

### Skill-Assessment Exercise 5.1

Find the equivalent transfer function $T(s) = C(s)/R(s)$ for the block diagram below.

![block diagram for skill-assessment exercise 5.1](./images/skill_exercise_5-1.png)

In [9]:
G1 = TransferFunction(s)
G2 = TransferFunction(s)
G3 = TransferFunction(1 / s)
G4 = TransferFunction(1 / s)
G5 = TransferFunction(s)

In [10]:
Ge1 = G1 * G2
Ge2 = Ge1 + G3
Ge3 = Ge2.feedback(TransferFunction(1))
Ge4 = Ge3 * G4
Ge5 = Ge4.feedback(G5)
Ge5.expr

(0.5*s**3 + 0.5)/(1.0*s**4 + 0.5*s**2 + 1.0*s)

## 5.3 : Analysis and Design of Feedback Systems

### Example 5.3 : Finding Transient Response

Find the peak time, percent overshoot, and settling time for the system:

![feedback system for example 5.3](./images/example_5-3.png)

In [11]:
G = TransferFunction(25 / (s * (s + 5)))
T = G.feedback(TransferFunction(1))
T.expr

25.0/(1.0*s**2 + 5.0*s + 25.0)

In [12]:
system = SecondOrderSystem(a=5, b=25)
print(
    f"the system is {system.get_natural_response_type().value}",
    f"natural frequency = {system.omega_nat:.3f} rad/s",
    f"damping ratio = {system.zeta:.3f}",
    f"peak time = {system.peak_time:.3f} s",
    f"percent overshoot = {system.percent_overshoot:.3f} %",
    f"settling time = {system.settling_time:.3f} s",
    sep='\n'
)

the system is underdamped
natural frequency = 5.000 rad/s
damping ratio = 0.500
peak time = 0.726 s
percent overshoot = 16.303 %
settling time = 1.622 s


### Example 5.4 : Gain Design for Transient Response

Design the value of gain $K$ for the feedback control system, so that the system will respond with 10% overshoot.

![feedback system for example 5.4](./images/example_5-4.png)

In [13]:
system = SecondOrderSystem.from_design_specs(percent_overshoot=10, a=5)
print(f"the required gain is {system.b:.2f}")

the required gain is 17.88


## 5.5 : Mason's Rule

### Example 5.7 : Transfer Function via Mason's Rule

Find the transfer function $C(s)/R(s)$ for the signal-flow graph below.

![signal-flow graph for example 5.7](./images/example_5-7.png)

Define the branches in the signal-flow graph:

In [14]:
branches = [
    SignalFlowGraph.Branch('G1', sp.Function('G1')(s), 'R', 'V4'),
    SignalFlowGraph.Branch('G2', sp.Function('G2')(s), 'V4', 'V3'),
    SignalFlowGraph.Branch('G3', sp.Function('G3')(s), 'V3', 'V2'),
    SignalFlowGraph.Branch('G4', sp.Function('G4')(s), 'V2', 'V1'),
    SignalFlowGraph.Branch('G5', sp.Function('G5')(s), 'V1', 'C'),
    SignalFlowGraph.Branch('H1', sp.Function('H1')(s), 'V3', 'V4', feedback=True),
    SignalFlowGraph.Branch('H2', sp.Function('H2')(s), 'V1', 'V2', feedback=True),
    SignalFlowGraph.Branch('G6', sp.Function('G6')(s), 'C', 'V5', feedback=True),
    SignalFlowGraph.Branch('G7', sp.Function('G7')(s), 'V5', 'V6', feedback=True),
    SignalFlowGraph.Branch('G8', sp.Function('G8')(s), 'V6', 'V4', feedback=True),
    SignalFlowGraph.Branch('H4', sp.Function('H4')(s), 'V6', 'V5')
]

> **Note**<br>
Branches that point back to the input of the signal-flow graph need to be indicated as *feedback branches*. 

Create the signal-flow graph and add the branches:

In [15]:
graph = SignalFlowGraph('R', 'C')

for branch in branches: 
    graph.add_branch(branch)

In [16]:
print(
    f"forward paths: {graph.forward_paths}",
    f"loops: {graph.loops}",
    f"non-touching loop groups: {graph.non_touching_loop_groups()}",
    f"2-size non-touching loop combinations: "
    f"{graph.non_touching_loop_combinations(size=2)}",
    f"3-size non-touching loop combinations: "
    f"{graph.non_touching_loop_combinations(size=3)}",
    f"4-size non-touching loop combinations: "
    f"{graph.non_touching_loop_combinations(size=4)}",
    sep='\n'
)

forward paths: [G1 ->- G2 ->- G3 ->- G4 ->- G5]
loops: [G2 ->- G3 ->- G4 ->- G5 ->- G6 ->- G7 ->- G8, G2 ->- H1, G4 ->- H2, G7 ->- H4]
non-touching loop groups: [[G2 ->- H1, G4 ->- H2, G7 ->- H4]]
2-size non-touching loop combinations: [(G2 ->- H1, G4 ->- H2), (G2 ->- H1, G7 ->- H4), (G4 ->- H2, G7 ->- H4)]
3-size non-touching loop combinations: [(G2 ->- H1, G4 ->- H2, G7 ->- H4)]
4-size non-touching loop combinations: []


In [17]:
print(
    f"forward path gains: {graph.forward_path_gains()}",
    f"loop gains: {graph.loop_gains()}",
    f"non-touching loop combination gains: {graph.non_touching_loop_combination_gains()}",
    f"denominator of transfer function: {graph.denominator}",
    f"numerator of transfer function: {graph.numerator}",
    f"transfer function: {graph.transfer_function}",
    sep='\n'
)

forward path gains: [G1(s)*G2(s)*G3(s)*G4(s)*G5(s)]
loop gains: [G2(s)*G3(s)*G4(s)*G5(s)*G6(s)*G7(s)*G8(s), G2(s)*H1(s), G4(s)*H2(s), G7(s)*H4(s)]
non-touching loop combination gains: {2: [G2(s)*G4(s)*H1(s)*H2(s), G2(s)*G7(s)*H1(s)*H4(s), G4(s)*G7(s)*H2(s)*H4(s)], 3: [G2(s)*G4(s)*G7(s)*H1(s)*H2(s)*H4(s)]}
denominator of transfer function: -G2(s)*G3(s)*G4(s)*G5(s)*G6(s)*G7(s)*G8(s) - G2(s)*G4(s)*G7(s)*H1(s)*H2(s)*H4(s) + G2(s)*G4(s)*H1(s)*H2(s) + G2(s)*G7(s)*H1(s)*H4(s) - G2(s)*H1(s) + G4(s)*G7(s)*H2(s)*H4(s) - G4(s)*H2(s) - G7(s)*H4(s) + 1
numerator of transfer function: (-G7(s)*H4(s) + 1)*G1(s)*G2(s)*G3(s)*G4(s)*G5(s)
transfer function: (-G7(s)*H4(s) + 1)*G1(s)*G2(s)*G3(s)*G4(s)*G5(s)/(-G2(s)*G3(s)*G4(s)*G5(s)*G6(s)*G7(s)*G8(s) - G2(s)*G4(s)*G7(s)*H1(s)*H2(s)*H4(s) + G2(s)*G4(s)*H1(s)*H2(s) + G2(s)*G7(s)*H1(s)*H4(s) - G2(s)*H1(s) + G4(s)*G7(s)*H2(s)*H4(s) - G4(s)*H2(s) - G7(s)*H4(s) + 1)


### Skill-Assessment Exercise 5.4

Use Mason's rule to find the transfer function of the signal-flow diagram below. This is the same system which is also shown in the block diagram.

![simplied signal-flow graph](./images/skill_exercise_5-4.png)

![block diagram of example 5.2](./images/skill_exercise_5-4b.png)

In [18]:
branches = [
    SignalFlowGraph.Branch('U1', sp.Integer(1), 'R', 'V1'),  # R = V1
    SignalFlowGraph.Branch('G1', sp.Function('G1')(s), 'V1', 'V3'),
    SignalFlowGraph.Branch('G2', sp.Function('G2')(s), 'V3', 'V4'),
    SignalFlowGraph.Branch('U2', sp.Integer(1), 'V3', 'V5'),
    SignalFlowGraph.Branch('U3', sp.Integer(1), 'V4', 'V5'),
    SignalFlowGraph.Branch('G3', sp.Function('G3')(s), 'V5', 'C'),
    SignalFlowGraph.Branch('-H2', -sp.Function('H2')(s), 'V4', 'V3', feedback=True),
    SignalFlowGraph.Branch('-H3', -sp.Function('H3')(s), 'C', 'V5', feedback=True),
    SignalFlowGraph.Branch('-H1', -sp.Function('H1')(s), 'V4', 'V1', feedback=True)
]

In [19]:
graph = SignalFlowGraph('R', 'C')
for br in branches: 
    graph.add_branch(br)

In [20]:
print(
    f"forward paths: {graph.forward_paths}",
    f"loops: {graph.loops}",
    f"non-touching loop groups: {graph.non_touching_loop_groups()}",
    f"2-size non-touching loop combinations: "
    f"{graph.non_touching_loop_combinations(size=2)}",
    f"3-size non-touching loop combinations: "
    f"{graph.non_touching_loop_combinations(size=3)}",
    f"4-size non-touching loop combinations: "
    f"{graph.non_touching_loop_combinations(size=4)}",
    sep='\n'
)

forward paths: [U1 ->- G1 ->- G2 ->- U3 ->- G3, U1 ->- G1 ->- U2 ->- G3]
loops: [G3 ->- -H3, G2 ->- -H2, G1 ->- G2 ->- -H1]
non-touching loop groups: [[G3 ->- -H3, G2 ->- -H2], [G1 ->- G2 ->- -H1, G3 ->- -H3]]
2-size non-touching loop combinations: [(G3 ->- -H3, G2 ->- -H2), (G1 ->- G2 ->- -H1, G3 ->- -H3)]
3-size non-touching loop combinations: []
4-size non-touching loop combinations: []


In [21]:
print(
    f"forward path gains: {graph.forward_path_gains()}",
    f"loop gains: {graph.loop_gains()}",
    f"non-touching loop combination gains: {graph.non_touching_loop_combination_gains()}",
    f"numerator of transfer function: {graph.numerator.factor()}",
    f"denominator of transfer function: {graph.denominator.factor()}",
    f"transfer function: {graph.transfer_function.factor()}",
    sep='\n'
)

forward path gains: [G1(s)*G2(s)*G3(s), G1(s)*G3(s)]
loop gains: [-G3(s)*H3(s), -G2(s)*H2(s), -G1(s)*G2(s)*H1(s)]
non-touching loop combination gains: {2: [G2(s)*G3(s)*H2(s)*H3(s), G1(s)*G2(s)*G3(s)*H1(s)*H3(s)]}
numerator of transfer function: (G2(s) + 1)*G1(s)*G3(s)
denominator of transfer function: (G3(s)*H3(s) + 1)*(G1(s)*G2(s)*H1(s) + G2(s)*H2(s) + 1)
transfer function: (G2(s) + 1)*G1(s)*G3(s)/((G3(s)*H3(s) + 1)*(G1(s)*G2(s)*H1(s) + G2(s)*H2(s) + 1))
